In [ ]:
!pip install -q spacy
!pip install -q tensorflow
!pip install -q transformers

In [ ]:
%env TF_CPP_MIN_LOG_LEVEL = 3

env: TF_CPP_MIN_LOG_LEVEL=3


In [ ]:
import math
import nltk
import spacy
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import transformers
from tokenizers import BertWordPieceTokenizer
from tqdm import tqdm
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from tensorflow import keras
from keras.utils import to_categorical
from tensorflow.keras import Sequential
from keras.preprocessing.text import Tokenizer
from keras.metrics import Precision, Recall, AUC
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.callbacks import EarlyStopping, LearningRateScheduler, CallbackList, ReduceLROnPlateau
from tensorflow.keras.optimizers.experimental import Adam

# **Loading Data**

In [ ]:
# Run only if you are using Google Colab
from google.colab import drive

'''drive.mount('/content/drive')

import os

path = '/content/drive/MyDrive/Colab Notebooks/Cap7''''

path = './datas'

Mounted at /content/drive


In [ ]:
train_data = pd.read_csv(os.path.join(path, 'dados_treino.txt'), header = None, delimiter = ';')

test_data = pd.read_csv(os.path.join(path, 'dados_teste.txt'), header = None, delimiter = ';')

In [ ]:
train_data = train_data.rename(columns = {0: 'text', 1: 'sentiment'})
test_data = test_data.rename(columns = {0: 'text', 1: 'sentiment'})

In [ ]:
train_data.shape

(16000, 2)

In [ ]:
test_data.shape

(2000, 2)

In [ ]:
train_data.head()

,text,sentiment
0,i am feeling completely overwhelmed i have two...,fear
1,i have the feeling she was amused and delighted,joy
2,i was able to help chai lifeline with your sup...,joy
3,i already feel like i fucked up though because...,anger
4,i still love my so and wish the best for him i...,sadness


In [ ]:
train_data['sentiment'].value_counts()

sentiment
joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: count, dtype: int64

In [ ]:
test_data['sentiment'].value_counts()

sentiment
joy         695
sadness     581
anger       275
fear        224
love        159
surprise     66
Name: count, dtype: int64

# **PreProcessing with Spacy**

In [ ]:
# Download Dict
!python -m spacy download en_core_web_md -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 13.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Loading Dict
nlp_dict = spacy.load('en_core_web_md')

In [ ]:
def preprocessing_text(text):
  # Processing text with Dict
  doc = nlp_dict(text)

  # Create an list with tokens
  tokens = [token.lemma_.lower().strip() for token in doc if not token.is_stop]

  # Return tokens
  return ' '.join(tokens)

In [ ]:
train_data['Processed_text'] = train_data['text'].apply(preprocessing_text)

test_data['Processed_text'] = test_data['text'].apply(preprocessing_text)

In [ ]:
train_data.head()

,text,sentiment,Processed_text
0,i am feeling completely overwhelmed i have two...,fear,feel completely overwhelmed strategy help feel...
1,i have the feeling she was amused and delighted,joy,feeling amuse delight
2,i was able to help chai lifeline with your sup...,joy,able help chai lifeline support encouragement ...
3,i already feel like i fucked up though because...,anger,feel like fuck not usually eat morning
4,i still love my so and wish the best for him i...,sadness,love wish good long tolerate effect bm life fa...


In [ ]:
lstm_tokenizer = Tokenizer()

lstm_tokenizer.fit_on_texts(train_data['Processed_text'])

In [ ]:
word_index = lstm_tokenizer.word_index

In [ ]:
# Iterates over dictionary key-value pairs
for i, (chave, valor) in enumerate(word_index.items()):
    print(chave, valor)
    if i == 9:
        break

In [ ]:
# Convert text to tokens sequences
train_seq = lstm_tokenizer.texts_to_sequence(train_data['Processed_text'])

In [ ]:
# Max Length of Sequence
max_length = 100

In [ ]:
train_seq_pad = pad_sequences(train_seq, maxlen = max_length, truncating = 'post')

In [ ]:
test_seq = lstm_tokenizer.texts_to_sequence(test_data['Processed_text'])

test_seq_pad = pad_sequences(test_seq, maxlen = max_length, truncating = 'post')

In [ ]:
# convert text to Number to train the model
label_encoder = LabelEncoder()

y_train_le = label_encoder.fit_transform(train_data['sentiment'])

y_test_le = label_encoder.transform(test_data['sentiment'])

y_train_encoded = to_categorical(y_train_le)
y_test_encoded = to_categorical(y_test_le)

# **Creating Model**

In [ ]:
vocab_size = len(lstm_tokenizer.word_index) + 1

In [ ]:
embedding_dim = max_length

In [ ]:
model = tf.keras.Sequential([Embedding(vocab_size, embedding_dim, input_length = max_length),
                            Bidirectional(LSTM(64)),
                            Dropout(0.4),
                            Dense(32, activation = 'leaky_relu', kernel_regularizer = 'l1_l2'),
                            Dense(6, activation = 'softmax')])

In [ ]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = 'accuracy', Precision(), Recall(), AUC())

In [ ]:
model.summary()

In [ ]:
input_data = np.array(train_seq_pad)
output_data = np.array(y_train_encoded)

In [ ]:
num_epochs = 35
validation_split_value = 0.2
patience = 5

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience = patience)

In [ ]:
%%time
history = model.fit(input_data,
                    output_data,
                    epochs = num_epochs
                    verbose = 1,
                    validation_split = validation_split_value,
                    callbacks = [early_stopping])